In [18]:
import pandas as pd
from pathlib import Path

STORAGE_FOLDER = Path(".").resolve()

In [19]:
column_names = [
    "Transaction unique identifier",
    "Price",
    "Date of Transfer",
    "Postcode",
    "Property Type",
    "Old/New",
    "Duration",
    "PAON",
    "SAON",
    "Street",
    "Locality",
    "Town/City",
    "District",
    "County",
    "PPD Category Type",
    "Record Status",
]

df_2024 = pd.read_csv(STORAGE_FOLDER / "data" / "pp-2024.csv", header=None, names=column_names)
df_2024["year"] = 2024

df = df_2024.copy()

# Filter for only Category Type A (Standard Price Paid entries)
df = df[df["PPD Category Type"] == "A"]

df = df.reset_index(drop=True)

In [20]:
# Load postcode to constituency mapping
postcodes = pd.read_csv(STORAGE_FOLDER / "data" / "postcodes_with_con.csv")

In [21]:
postcodes[postcodes.short_code == "UKPARL.2025.HSP"].sample(5)

,postcode,short_code,__index_level_0__
1080364,NW1 0XE,UKPARL.2025.HSP,159
1080248,NW1 0EH,UKPARL.2025.HSP,43
1080711,NW1 3HB,UKPARL.2025.HSP,506
1084874,NW1W 7SW,UKPARL.2025.HSP,4669
1654717,WC2B 5LX,UKPARL.2025.HSP,1733


In [22]:
postcodes.postcode = postcodes.postcode.str.replace(" ", "")
df["constituency"] = df.Postcode.str.replace(" ", "").map(postcodes.set_index("postcode").short_code)

In [23]:
df[df.constituency == "UKPARL.2025.HSP"].Price.median() # https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/parliamentaryconstituencyhousepricestatisticsforsmallareas gets 676k in March 2025

np.float64(700000.0)

In [24]:
(df[df.constituency == "UKPARL.2025.HSP"].Price >= 1.75e6).mean()

np.float64(0.1141439205955335)

In [25]:
import numpy as np

bins = [0, 250000, 500000, 750000, 1000000, 1500000, 1750000, 2000000, 3000000, 5000000, 10000000, np.inf]
counts = []

for i in range(len(bins) - 1):
    lower = bins[i]
    upper = bins[i + 1]
    count = (df[df.constituency == "UKPARL.2025.HSP"].Price >= lower) & (df[df.constituency == "UKPARL.2025.HSP"].Price < upper)
    counts.append(count.mean())

stats = pd.DataFrame({
    "bin_lower": bins[:-1],
    "bin_upper": bins[1:],
    "proportion": counts
})

In [26]:
df[df.constituency == "UKPARL.2025.HSP"].groupby("year").Price.count()

year
2024    806
Name: Price, dtype: int64

In [27]:
stats.bin_lower = stats.bin_lower.astype(str)

stats["text"] = stats.proportion.apply(lambda x: f"{x:.1%}")
stats["bin_lower_text"] = stats.bin_lower.apply(lambda x: f"£{int(x):,}")

import plotly.express as px

px.bar(stats, x="bin_lower_text", y="proportion", title="Proportion of houses sold in each price band in Holborn and St Pancras", text="text").update_layout(
    height=600,
    width=800,
    xaxis_title="Price band lower bound",
    yaxis_title="Proportion of houses sold",
)

In [28]:
import plotly.express as px

fig = px.histogram(df[df.constituency == "UKPARL.2025.HSP"].Price, nbins=100, cumulative=True)

fig.update_layout(
    title="Distribution of sale prices in Holborn and St Pancras, 2026 forecast prices",
    height=600,
    width=800,
    showlegend=False,
)

In [29]:
# Count sales in Holborn and St Pancras
hsp_sales = df[df.constituency == "UKPARL.2025.HSP"]
print(f"Total sales in Holborn and St Pancras (2024): {len(hsp_sales)}")
print(f"By year breakdown:")
print(hsp_sales.groupby("year").Price.count())

Total sales in Holborn and St Pancras (2024): 806
By year breakdown:
year
2024    806
Name: Price, dtype: int64
